In [1]:
from crossreference import *
import chart.johncartersqueeze as jcsqueeze

In [2]:
import pandas as pd
import numpy as np
import datetime

In [3]:
import mysql.connector

def load_stockdata(symbol):
    serverName = "localhost"
    cnx = mysql.connector.connect(user='michael', password='Password2017',
                                  host=serverName, database='stock_market')
    cursor = cnx.cursor()
    
    query = \
    "SELECT stock.*, bb.`sma` AS bb_middle, bb.`band_upper` AS bb_upper, bb.`band_lower` AS bb_lower, \
    kc.`ema20` AS kc_middle, kc.`atr2_upper` AS kc_2_upper, kc.`atr2_lower` AS kc_2_lower, \
    kc.`atr3_upper` AS kc_3_upper, kc.`atr3_lower` AS kc_3_lower, \
    kc.`atr1_upper` AS kc_1_upper, kc.`atr1_lower` AS kc_1_lower, \
    macd.`macd_histogram` as macd, ttma.macd_histogram as wave_a, ttmb.macd_histogram as wave_b, ttmc.macd_histogram as wave_c \
    FROM \
    ( SELECT * FROM (SELECT symbol, effective_date, `adj_open`, `adj_high`, `adj_low`, `adj_close`, `adj_volume` FROM `market.stock_price` \
      WHERE symbol = '{0}' ORDER BY effective_date DESC LIMIT 180) sub ORDER BY `effective_date` ASC ) stock \
    LEFT JOIN \
    `idc.bollinger_bands` AS bb ON bb.symbol = stock.symbol AND bb.effective_date = stock.effective_date \
    LEFT JOIN \
    `idc.keltner_channels` AS kc ON kc.symbol = stock.symbol AND kc.effective_date = stock.effective_date \
    LEFT JOIN \
    `idc.macd` as macd on macd.symbol = stock.symbol and macd.effective_date = stock.effective_date \
        and macd.short_period = 12 and macd.long_period=26 and macd.signal_period=9 \
    LEFT JOIN \
    `idc.macd` as ttma on ttma.symbol = stock.symbol and ttma.effective_date = stock.effective_date \
        and ttma.short_period = 8 and ttma.long_period=34 and ttma.signal_period=34 \
    LEFT JOIN \
    `idc.macd` as ttmb on ttmb.symbol = stock.symbol and ttmb.effective_date = stock.effective_date \
        and ttmb.short_period = 8 and ttmb.long_period=89 and ttmb.signal_period=89 \
    LEFT JOIN \
    `idc.macd` as ttmc on ttmc.symbol = stock.symbol and ttmc.effective_date = stock.effective_date \
        and ttmc.short_period = 8 and ttmc.long_period=144 and ttmc.signal_period=144 \
    ".format(symbol)

    cursor.execute(query)
    dfsql = pd.DataFrame(cursor.fetchall(), dtype=np.float)
    dfsql.columns = cursor.column_names
    dfsql.set_index('effective_date', inplace=True)
    cursor.close()
    return dfsql


In [10]:
df = load_stockdata('AMZN')
# df['macd_up'] = df.macd > df.macd.shift(-1)
#df['squeeze'] = (df.bb_lower >= df.kc_2_lower) & (df.bb_upper <= df.kc_2_upper)
df.head(20)


,symbol,adj_open,adj_high,adj_low,adj_close,adj_volume,bb_middle,bb_upper,bb_lower,kc_middle,kc_2_upper,kc_2_lower,kc_3_upper,kc_3_lower,kc_1_upper,kc_1_lower,macd,wave_a,wave_b,wave_c
effective_date,,,,,,,,,,,,,,,,,,,,
2018-02-23,AMZN,1495.34,1500.00,1486.50,1500.00,4327008.0,1428.7085,1512.635820,1344.781180,1419.910872,1509.537008,1330.284736,1554.350076,1285.471668,1464.723940,1375.097804,2.590641,NaN,NaN,NaN
2018-02-22,AMZN,1495.36,1502.54,1475.76,1484.76,4732555.0,1422.6060,1502.566130,1342.645870,1411.480438,1505.656278,1317.304598,1552.744198,1270.216678,1458.568358,1364.392518,1.586442,NaN,NaN,NaN
2018-02-21,AMZN,1485.00,1503.49,1478.92,1482.92,6216694.0,1416.2435,1495.657420,1336.829580,1403.766800,1501.066934,1306.466666,1549.717001,1257.816599,1452.416867,1355.116733,0.803004,NaN,NaN,NaN
2018-02-20,AMZN,1446.49,1488.77,1446.49,1468.35,6388374.0,1410.2245,1486.705344,1333.743656,1395.434884,1494.813490,1296.056278,1544.502793,1246.366975,1445.124187,1345.745581,-0.921287,NaN,NaN,NaN
2018-02-16,AMZN,1457.37,1465.80,1446.56,1448.69,4410879.0,1403.1725,1482.857158,1323.487842,1387.759608,1488.278108,1287.241108,1538.537358,1236.981858,1438.018858,1337.500358,-2.703390,NaN,NaN,NaN
2018-02-15,AMZN,1466.89,1468.94,1436.84,1461.76,5598111.0,1395.4670,1485.223578,1305.710422,1381.345883,1486.636575,1276.055191,1539.281921,1223.409845,1433.991229,1328.700537,-3.913777,2.184721,69.166973,122.680912
2018-02-14,AMZN,1406.25,1452.06,1403.36,1451.05,5881238.0,1387.0450,1481.809828,1292.280172,1372.881239,1481.332753,1264.429725,1535.558510,1210.203968,1427.106996,1318.655482,-7.453123,-3.070768,63.491592,116.078128
2018-02-13,AMZN,1385.93,1419.72,1383.53,1414.51,5858860.0,1379.2425,1477.283232,1281.201768,1364.652949,1473.954579,1255.351319,1528.605394,1200.700504,1419.303764,1310.002134,-12.005196,-9.400855,57.020688,108.748149
2018-02-12,AMZN,1364.67,1393.81,1344.01,1386.23,6682214.0,1373.7600,1475.492718,1272.027282,1359.404838,1471.546594,1247.263082,1527.617472,1191.192204,1415.475716,1303.333960,-10.716940,-11.224673,56.370610,107.542525


In [11]:
jcsqueeze.drawchart('AMZN', df)

In [6]:
import repository.stockpricerepo as pricerepo

In [7]:
df = pricerepo.get_stock_adjdata('ALB', datetime.date(2017,1,1), datetime.date(2018,2,20))
df.head()

NameError: name 'pricerepo' is not defined

In [6]:
import repository.simpleindicatorrepo as pricerepo

ImportError: cannot import name 'indicatorrepo'